In [3]:
!pip install pandas-datareader

  Obtaining dependency information for pandas-datareader from https://files.pythonhosted.org/packages/3f/16/56c9d648b503619ebe96f726b5f642b68e299b34162ed2d6faa9d7966b7d/pandas_datareader-0.10.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/109.5 kB ? eta -:--:--
   ------------------------------------- -- 102.4/109.5 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 109.5/109.5 kB 2.1 MB/s eta 0:00:00


In [3]:
import yfinance as yf
import pandas as pd
import pandas_datareader.data as web
import numpy as np

# -------------------------------------
# 1. BEÁLLÍTÁSOK
# -------------------------------------
tickers = [
    "AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "AMD", "AMGN", "AMT", "AMZN",
    "AVGO", "AXP", "BA", "BAC", "BK", "BKNG", "BLK", "BMY", "BRK-B", "C",
    "CAT", "CL", "CMCSA", "COF", "COP", "COST", "CRM", "CSCO", "CVS", "CVX",
    "DE", "DHR", "DIS", "DUK", "EMR", "FDX", "GD", "GE", "GILD", "GM",
    "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "INTU", "ISRG", "JNJ",
    "JPM", "KO", "LIN", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT",
    "MET", "META", "MMM", "MO", "MRK", "MS", "MSFT", "NEE", "NFLX", "NKE",
    "NOW", "NVDA", "ORCL", "PEP", "PFE", "PG", "PLTR", "PM", "PYPL", "QCOM",
    "RTX", "SBUX", "SCHW", "SO", "SPG", "T", "TGT", "TMO", "TMUS", "TSLA",
    "TXN", "UBER", "UNH", "UNP", "UPS", "USB", "V", "VZ", "WFC", "WMT", "XOM"
]

# Kért időszak beállítása
start_date = "2002-01-01"
end_date = "2025-11-01"

print("--- ADATOK LETÖLTÉSE (CAPM ELŐKÉSZÍTÉS) ---")
print(f"Időszak: {start_date} -> {end_date}")

try:
    # -------------------------------------
    # 2. RÉSZVÉNYEK ÉS PIAC (YAHOO)
    # -------------------------------------
    print("\n1. Részvények és Piaci index (^OEX) letöltése...")
    
    # Részvények letöltése
    stocks_raw = yf.download(tickers, start=start_date, end=end_date, interval="1d", auto_adjust=False, threads=True)
    
    # Árfolyam kiválasztása (Adj Close preferált, ha nincs, akkor Close)
    if 'Adj Close' in stocks_raw.columns:
        stock_prices = stocks_raw['Adj Close']
    else:
        stock_prices = stocks_raw['Close']
    
    # Piac (S&P 100 - OEX) letöltése
    market_raw = yf.download("^OEX", start=start_date, end=end_date, interval="1d", auto_adjust=False, progress=False)
    market_price = market_raw['Adj Close'] if 'Adj Close' in market_raw.columns else market_raw['Close']
    
    # -------------------------------------
    # 3. KOCKÁZATMENTES HOZAM (FRED 1M T-Bill)
    # -------------------------------------
    print("2. Kockázatmentes hozam (FRED: DGS1MO - 1-Month Treasury)...")
    
    # DGS1MO: Market Yield on U.S. Treasury Securities at 1-Month Constant Maturity
    # A FRED adatok százalékban vannak (pl. 4.5 jelentése 4.5%)
    rf_data = web.DataReader("DGS1MO", "fred", start=start_date, end=end_date)
    
    # -------------------------------------
    # 4. FELDOLGOZÁS ÉS ÖSSZEFŰZÉS
    # -------------------------------------
    print("3. Adatok feldolgozása és mentése...")

    # Napi hozamok számítása a részvényeknél és a piacnál
    stock_returns = stock_prices.pct_change().dropna(axis=0, how='all')
    market_return = market_price.pct_change().dropna()
    market_return.name = "Market_Return"

    # Rf feldolgozása:
    # 1. Hiányzó adatok pótlása (előző napi értékkel, mert ünnepnapokon nincs adat a FRED-en)
    rf_filled = rf_data.ffill()
    
    # 2. Konverzió: Százalék -> Tizedes -> Napi ráta
    # Képlet: r_daily = r_annual / 100 / 252
    rf_daily = rf_filled / 100 / 252
    rf_daily.columns = ["Risk_Free_Rate"]

    # Indexek igazítása (Yahoo időzónás, FRED nem - levágjuk a zónát)
    stock_returns.index = stock_returns.index.tz_localize(None)
    market_return.index = market_return.index.tz_localize(None)
    # A FRED indexe már timezone-naive datetime, de biztosítjuk a közös metszetet
    
    # Csak azokat a napokat tartjuk meg, ahol minden adat rendelkezésre áll (inner join)
    # Először összefűzzük a piacot és az Rf-et
    factors = pd.concat([market_return, rf_daily], axis=1).dropna()
    
    # Majd összefűzzük a részvényekkel
    final_data = stock_returns.join(factors, how='inner')

    # -------------------------------------
    # 5. MENTÉS
    # -------------------------------------
    # 1. CSV a hozamokkal (CAPM-hez ez kell)
    final_data.to_csv('capm_data_returns.csv')
    
    # 2. CSV a nyers árakkal (ha külön kérted)
    stock_prices.to_csv('stock_prices_raw.csv')

    print(f"\nSIKER! Adatok elmentve:")
    print(f" -> 'capm_data_returns.csv' (Mérete: {final_data.shape}) - Ezt használd a regresszióhoz.")
    print(f" -> 'stock_prices_raw.csv' - Nyers árfolyamok.")

except Exception as e:
    print(f"\nHIBA TÖRTÉNT: {e}")

--- ADATOK LETÖLTÉSE (CAPM ELŐKÉSZÍTÉS) ---
Időszak: 2002-01-01 -> 2025-11-01

1. Részvények és Piaci index (^OEX) letöltése...


[*********************100%***********************]  101 of 101 completed


2. Kockázatmentes hozam (FRED: DGS1MO - 1-Month Treasury)...
3. Adatok feldolgozása és mentése...

SIKER! Adatok elmentve:
 -> 'capm_data_returns.csv' (Mérete: (5997, 103)) - Ezt használd a regresszióhoz.
 -> 'stock_prices_raw.csv' - Nyers árfolyamok.


In [6]:
import yfinance as yf
import pandas as pd
import pandas_datareader.data as web
import numpy as np

# -------------------------------------
# 1. BEÁLLÍTÁSOK (SP100 Lista)
# -------------------------------------
tickers = [
    "AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "AMD", "AMGN", "AMT", "AMZN",
    "AVGO", "AXP", "BA", "BAC", "BK", "BKNG", "BLK", "BMY", "BRK-B", "C",
    "CAT", "CL", "CMCSA", "COF", "COP", "COST", "CRM", "CSCO", "CVS", "CVX",
    "DE", "DHR", "DIS", "DUK", "EMR", "FDX", "GD", "GE", "GILD", "GM",
    "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "INTU", "ISRG", "JNJ",
    "JPM", "KO", "LIN", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT",
    "MET", "META", "MMM", "MO", "MRK", "MS", "MSFT", "NEE", "NFLX", "NKE",
    "NOW", "NVDA", "ORCL", "PEP", "PFE", "PG", "PLTR", "PM", "PYPL", "QCOM",
    "RTX", "SBUX", "SCHW", "SO", "SPG", "T", "TGT", "TMO", "TMUS", "TSLA",
    "TXN", "UBER", "UNH", "UNP", "UPS", "USB", "V", "VZ", "WFC", "WMT", "XOM"
]

start_date = "2002-01-01"
end_date = "2025-11-01"

print("--- ADATOK PÓTLÁSA (HAVI + NAPI) ---")

try:
    # ==========================================
    # A) HAVI ADATOK LETÖLTÉSE (1. és 2. Feladat)
    # ==========================================
    print("\n1. HAVI adatok letöltése és feldolgozása...")
    
    # Részvények (Havi)
    stocks_m = yf.download(tickers, start=start_date, end=end_date, interval="1mo", auto_adjust=False, threads=True)
    if 'Adj Close' in stocks_m.columns:
        prices_m = stocks_m['Adj Close']
    else:
        prices_m = stocks_m['Close']
        
    # Piaci Index (^OEX) (Havi)
    market_m = yf.download("^OEX", start=start_date, end=end_date, interval="1mo", auto_adjust=False, progress=False)
    mkt_price_m = market_m['Adj Close'] if 'Adj Close' in market_m.columns else market_m['Close']

    # Kockázatmentes hozam (FRED GS1M - Havi)
    print("   -> Kockázatmentes hozam (FRED)...")
    rf_data = web.DataReader("GS1M", "fred", start=start_date, end=end_date)

    # Feldolgozás (Hozamok)
    stock_ret_m = prices_m.pct_change().dropna(how='all')
    market_ret_m = mkt_price_m.pct_change().dropna()
    market_ret_m.name = "Market_Return" # Vagy "^OEX"
    
    # RF konverzió (Éves % -> Havi ráta)
    rf_ret_m = rf_data / 100 / 12
    rf_ret_m.columns = ["Risk_Free_Rate"]

    # Dátum szinkronizálás (Hónap szinten)
    stock_ret_m.index = stock_ret_m.index.to_period('M')
    market_ret_m.index = market_ret_m.index.to_period('M')
    rf_ret_m.index = rf_ret_m.index.to_period('M')

    # Összefűzés
    monthly_data = stock_ret_m.join([market_ret_m, rf_ret_m], how='inner')
    
    # Visszaalakítás Timestamp-re a mentéshez
    monthly_data.index = monthly_data.index.to_timestamp()
    
    # Mentés
    monthly_data.to_csv('capm_data_monthly.csv')
    print("   -> SIKER: 'capm_data_monthly.csv' elmentve.")

    # ==========================================
    # B) NAPI ADATOK LETÖLTÉSE (3. Feladat - Volatilitás)
    # ==========================================
    print("\n2. NAPI adatok letöltése és feldolgozása...")
    
    # Részvények (Napi)
    stocks_d = yf.download(tickers, start=start_date, end=end_date, interval="1d", auto_adjust=False, threads=True)
    if 'Adj Close' in stocks_d.columns:
        prices_d = stocks_d['Adj Close']
    else:
        prices_d = stocks_d['Close']

    # Hozam számítás
    daily_returns = prices_d.pct_change().dropna(how='all')
    
    # Időzóna tisztítás
    daily_returns.index = daily_returns.index.tz_localize(None)
    
    # Mentés
    daily_returns.to_csv('daily_stock_returns.csv')
    print("   -> SIKER: 'daily_stock_returns.csv' elmentve.")

    print("\n--- MINDEN ADAT KÉSZEN ÁLL ---")
    print("Most már lefuttathatod az összes feladat kódját (Momentum, Crash, Volatilitás).")

except Exception as e:
    print(f"\nHIBA TÖRTÉNT: {e}")

--- ADATOK PÓTLÁSA (HAVI + NAPI) ---

1. HAVI adatok letöltése és feldolgozása...


[*********************100%***********************]  101 of 101 completed


   -> Kockázatmentes hozam (FRED)...
   -> SIKER: 'capm_data_monthly.csv' elmentve.

2. NAPI adatok letöltése és feldolgozása...


[*********************100%***********************]  101 of 101 completed


   -> SIKER: 'daily_stock_returns.csv' elmentve.

--- MINDEN ADAT KÉSZEN ÁLL ---
Most már lefuttathatod az összes feladat kódját (Momentum, Crash, Volatilitás).


In [6]:
import yfinance as yf
import pandas as pd
import pandas_datareader.data as pdr
import datetime

# --- BEÁLLÍTÁSOK ---
start_date = '2002-01-01'
end_date = '2025-11-01'

tickers = [
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'META', 'TSLA', 'NVDA', 'BRK-B', 'JPM',
    'JNJ', 'V', 'PG', 'MA', 'HD', 'UNH', 'BAC', 'XOM', 'PFE', 'KO', 'DIS', 'CSCO',
    'PEP', 'ABBV', 'CVX', 'AVGO', 'MRK', 'WFC', 'COST', 'TMO', 'ABT', 'LLY', 'ADBE',
    'MCD', 'CRM', 'NKE', 'DHR', 'VZ', 'ACN', 'NEE', 'LIN', 'TXN', 'PM', 'BMY', 'AMD',
    'RTX', 'WMT', 'HON', 'UNP', 'MS', 'LOW', 'QCOM', 'INTC', 'COP', 'IBM', 'SPGI',
    'UPS', 'CAT', 'ORCL', 'GE', 'GS', 'AMAT', 'DE', 'BLK', 'BA', 'INTU', 'MMM', 'T',
    'PLD', 'AXP', 'ISRG', 'SYK', 'C', 'BKNG', 'EL', 'MDLZ', 'LMT', 'TJX', 'ADI',
    'GILD', 'ADP', 'MMC', 'CB', 'NOW', 'SCHW', 'ZTS', 'TMUS', 'MO', 'CVS', 'EOG',
    'REGN', 'PGR', 'VRTX', 'BSX', 'SO', 'BDX', 'CI', 'CL', 'DUK', 'ITW', 'SLB'
]

print("--- ADATLETÖLTÉS INDÍTÁSA")

# 1. LETÖLTÉS (Yahoo Finance - auto_adjust=False)
print("Részvényárak letöltése...")
stock_data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False)['Adj Close']

print("Piaci index (SP100) letöltése...")
market_data = yf.download('^OEX', start=start_date, end=end_date, auto_adjust=False)['Adj Close']

print("Kockázatmentes hozam (DGS1MO) letöltése...")
rf_raw = pdr.get_data_fred('DGS1MO', start=start_date, end=end_date).ffill()

# 2. ADATFELDOLGOZÁS (Havi adatok)

# A) Részvények (Monthly Stock Returns)
# 'M' resampling ('ME' helyett a kompatibilitás miatt)
monthly_prices = stock_data.resample('M').last()
# FONTOS: Itt nincs .dropna()! Megtartjuk az összes oszlopot, akkor is, ha NaN van benne.
monthly_stock_returns = monthly_prices.pct_change()

# B) Piac (Monthly Market Returns)
monthly_market = market_data.resample('M').last()
monthly_market_returns = monthly_market.pct_change()
monthly_market_returns.name = 'Market_Return' # Oszlopnév beállítása

# C) Kockázatmentes (Monthly Risk Free Rate)
rf_monthly = rf_raw.resample('M').last()
rf_monthly = rf_monthly / 100 / 12  # Százalék -> Tizedes, Éves -> Havi
rf_monthly.columns = ['Risk_Free_Rate']

# 3. MENTÉS KÜLÖN FÁJLOKBA

print("Mentés külön fájlokba...")

# --- A) HAVI ADATOK ---
monthly_stock_returns.to_csv('monthly_stock_returns.csv')
monthly_market_returns.to_csv('monthly_market_returns.csv')
rf_monthly.to_csv('monthly_risk_free_rate.csv')

# --- B) NAPI ADATOK (Szorgalmi feladathoz / Volatilitáshoz) ---

# 1. Napi Részvényhozamok (már volt, de dropna nélkül hagyjuk)
stock_data.pct_change().to_csv('daily_stock_returns.csv')

# 2. Napi Piaci Hozam (ÚJ: ezt is elmentjük külön)
daily_market = market_data.pct_change().dropna()
daily_market.name = 'Market_Return'
daily_market.to_csv('daily_market_returns.csv')

# 3. Napi Kockázatmentes Hozam (ÚJ: ezt kérted most)
# Ez a DGS1MO napi szinten, 252-vel osztva
rf_daily.to_csv('daily_risk_free_rate.csv')

# 4. Nyers árak (ellenőrzéshez)
stock_data.to_csv('stock_prices_raw.csv')

print("\n--- KÉSZ! ---")
print("Létrehozott HAVI fájlok:")
print(" - monthly_stock_returns.csv")
print(" - monthly_market_returns.csv")
print(" - monthly_risk_free_rate.csv (RF)")
print("\nLétrehozott NAPI fájlok:")
print(" - daily_stock_returns.csv")
print(" - daily_market_returns.csv")
print(" - daily_risk_free_rate.csv (RF - DGS1MO)")

--- ADATLETÖLTÉS INDÍTÁSA
Részvényárak letöltése...


[*********************100%***********************]  101 of 101 completed


Piaci index (SP100) letöltése...


[*********************100%***********************]  1 of 1 completed


Kockázatmentes hozam (DGS1MO) letöltése...
Mentés külön fájlokba...

--- KÉSZ! ---
Létrehozott HAVI fájlok:
 - monthly_stock_returns.csv
 - monthly_market_returns.csv
 - monthly_risk_free_rate.csv (RF)

Létrehozott NAPI fájlok:
 - daily_stock_returns.csv
 - daily_market_returns.csv
 - daily_risk_free_rate.csv (RF - DGS1MO)
